# Calculate normalised enrichment scores of ConsensusTME gene sets

Script related to figure 3a-d, s3c-f

### Aim:
- Calculate NES of ConsensusTME Gene Sets

In [1]:
sessionInfo()

R version 3.5.0 (2018-04-23)
Platform: x86_64-apple-darwin15.6.0 (64-bit)
Running under: macOS High Sierra 10.13.6

Matrix products: default
BLAS: /Library/Frameworks/R.framework/Versions/3.5/Resources/lib/libRblas.0.dylib
LAPACK: /Library/Frameworks/R.framework/Versions/3.5/Resources/lib/libRlapack.dylib

locale:
[1] C/UTF-8/C/C/C/C

attached base packages:
[1] stats     graphics  grDevices utils     datasets  methods   base     

loaded via a namespace (and not attached):
 [1] compiler_3.5.0  IRdisplay_0.6.1 pbdZMQ_0.3-3    tools_3.5.0    
 [5] htmltools_0.3.6 base64enc_0.1-3 crayon_1.3.4    Rcpp_1.0.1     
 [9] uuid_0.1-2      IRkernel_0.8.14 jsonlite_1.6    digest_0.6.18  
[13] repr_0.17       evaluate_0.13  

## Load packages

In [1]:
library(GSVA)
library(BBmisc)
library(nortest)

Warning message:
“package ‘GSVA’ was built under R version 3.5.1”
Attaching package: ‘BBmisc’

The following object is masked from ‘package:base’:

    isFALSE



In [2]:
setwd("~/git_repos/HGSOC_TME_Heterogeneity/Scripts/3/")

## Load expression data

In [ ]:
mix <- "../../Data/1/TreatmentNaive_log2exp_loess_norm.txt"

In [3]:
data <- read.csv(mix,
                 sep='\t',
                 header=T,
                 row.names='Hugo_Symbol')

In [4]:
head(data)

,A01,A02,A03,A04,A05,A06,A07,A08,A09,A10,⋯,C06,C07,C08,C09,C10,C11,C12,D01,D02,D03
A1BG,4.124711,3.880633,4.864973,4.167882,4.317845,3.679057,4.092837,4.370571,4.252637,4.174694,⋯,3.804537,4.416981,4.169014,4.503313,4.146333,4.078398,4.011369,3.845128,3.600927,4.521857
A2BP1,3.960924,4.033715,3.721251,3.916073,4.338194,3.357396,3.872447,3.803388,3.484735,4.427405,⋯,4.991068,3.799913,4.117883,4.122134,4.221760,3.648326,3.675317,3.546925,3.788279,4.920841
A2M,12.603155,10.859838,12.612923,9.176999,10.548790,11.840776,7.200547,11.835658,11.535547,7.080118,⋯,9.061598,9.189459,8.844728,7.464440,6.726233,9.265911,10.740486,9.735582,10.781599,8.734304
A2ML1,3.945877,4.000621,3.767932,3.789729,3.543204,3.805520,4.336280,4.031757,4.166018,3.673146,⋯,4.231174,3.871651,3.287232,3.721093,3.723801,3.884499,3.694111,3.937682,3.600471,3.402861
A4GALT,3.597277,3.863757,4.051844,4.113169,3.975144,3.991293,5.439960,4.696177,4.171788,4.041633,⋯,4.183089,4.699729,4.019537,3.739865,4.591953,4.997860,4.099630,5.001239,4.714710,4.145119
A4GNT,3.714481,3.593169,3.241352,3.515850,3.364447,3.767865,3.768695,3.163977,3.423377,3.205154,⋯,3.896812,3.915653,3.351422,3.719759,3.641028,3.114521,3.560199,3.539738,3.760903,3.479266


In [5]:
data_mtx <- as.matrix(data)

## Load ConsensusTME gene sets

In [6]:
ConTME_GeneSets <- read.csv('../../Data/3/ConsensusTME_GeneSets.txt',
                            sep='\t',
                            header=T)

In [7]:
head(ConTME_GeneSets)

B_cells,Cytotoxic_cells,Dendritic_cells,Endothelial,Eosinophils,Fibroblasts,Macrophages,Macrophages_M1,Macrophages_M2,Mast_cells,Monocytes,NK_cells,Neutrophils,Plasma_cells,T_cells_CD4,T_cells_CD8,T_cells_gamma_delta,T_regulatory_cells
BLNK,CTSW,BCL2A1,CD93,ABHD2,ABCA6,ACTR2,ACTR2,AIF1,ATP6V1C1,AIF1,APOBEC3G,BCL2A1,CD27,AIM2,BCL11B,APOBEC3G,CCR7
CD1C,GZMA,BIRC3,CDH5,BCL2A1,ASPN,ATG7,CCL19,ATP6V1C1,CCL4,AKAP13,CCL4,C5AR1,CD38,ARHGAP15,BIN2,CCL4,CD2
CD37,GZMB,CCL19,EMCN,CCL4,COL3A1,ATP6V1C1,CCL4,CLEC10A,CD84,C5AR1,CCR5,CSF2RB,CYBA,BATF,CCL4,CCR2,CD247
CD53,GZMH,CCL4,ERG,CCR1,DCN,BCL2A1,CCR1,CLEC4A,LCP2,CCR2,CD2,CSF3R,ENTPD1,BCL11B,CCR5,CCR5,CD27
CD69,KLRB1,CCR2,KDR,CD69,GREM1,C5AR1,CCR7,FGR,MMP9,CD244,CD244,FCGR3B,HSPA6,CCL4,CCR7,CD2,CD37
CD80,KLRK1,CCR5,PCDH12,CSF2RB,ISLR,CCL19,CD38,HCK,SAMSN1,CD37,CD247,FPR1,MTDH,CCR2,CD2,CD244,CD3E


In [8]:
ConTME_GeneSets <- convertRowsToList(t(ConTME_GeneSets))

## Run ssGSEA

In [9]:
con_NES <- gsva(data_mtx,
                ConTME_GeneSets,
                method='ssgsea',
                min.sz=0,
                max.sz=1000,
                ssgsea.norm=T)

Estimating ssGSEA scores for 18 gene sets.
  |                                                                      |   0%Using parallel with 4 cores
  |======================================================================| 100%


In [10]:
con_NES <- data.frame("Term"=rownames(con_NES),
                      con_NES)

rownames(con_NES) <- NULL

In [11]:
head(con_NES)

Term,A01,A02,A03,A04,A05,A06,A07,A08,A09,⋯,C06,C07,C08,C09,C10,C11,C12,D01,D02,D03
B_cells,0.03847931,0.1549990,0.1315862,0.08862296,0.03877229,-0.01751238,-0.04707827,-0.086647414,-0.19937432,⋯,0.01250423,0.11489017,-0.03624780,0.01627768,-0.01288170,0.14474920,0.12045088,0.10452696,0.08315162,-0.065880578
Cytotoxic_cells,0.21014819,0.1875674,0.1536875,-0.14416349,0.00385764,-0.15517350,-0.29025097,-0.170453505,-0.41685911,⋯,-0.34590063,-0.25087812,-0.30483075,-0.22186809,-0.30561410,-0.00397126,-0.00266926,-0.07507192,-0.08618847,-0.243335223
Dendritic_cells,0.18909641,0.2267871,0.2752798,0.04431845,0.09400068,0.09265174,-0.01060890,0.014540237,-0.05380443,⋯,0.02497896,0.13393122,0.03094155,0.07584708,0.02380026,0.16876260,0.15166472,0.14329850,0.21183219,0.002961015
Endothelial,0.15780636,0.1632387,0.2085353,0.01686532,0.03234967,0.05249416,-0.08634282,0.001588116,-0.04508628,⋯,0.03781560,0.16182852,0.08495025,0.09548239,-0.07743838,0.13262528,0.16642132,0.09170046,0.11939875,-0.119858918
Eosinophils,0.20082325,0.1976900,0.2719497,-0.02236162,0.08303401,0.07901006,-0.05555490,-0.011277352,-0.04878391,⋯,0.02971066,0.08064865,-0.04818705,0.13406243,0.03213819,0.16638669,0.15893988,0.20794872,0.20548346,-0.054183279
Fibroblasts,0.13261514,0.2166350,0.2155019,0.01534575,0.09724308,0.16575105,0.02957739,0.003008653,0.10290596,⋯,0.32002944,0.50893683,0.50814238,0.28581643,0.15155832,0.26258405,0.25894289,0.24088605,0.19288074,0.098543741


### Save NES

In [12]:
write.table(con_NES,
            file='../../Data/3/ConsensusTME_TreatmentNaive_NES.txt',
            sep='\t',
            row.names=F,
            col.names=T)

# End script